In [1]:
# set up the model
setwd('/lustre/scratch117/cellgen/team297/kt16/newcastle_covid')
library(dplyr)

bcell.df <- read.csv("h5ad/combined_dec_bcells_obs.csv")

covid.meta <- bcell.df[,c('sample_id', 'patient_id', 'Status_on_day_collection_summary', 'Age', 'Sex', 'Collection_Day')] %>% distinct()
rownames(covid.meta) <- covid.meta$sample_id

bcell.meta <- covid.meta[!covid.meta$Status_on_day_collection_summary %in% c("Non_covid", "LPS"),]
bcell.meta$OrderedSeverity <- ordered(bcell.meta$Status_on_day_collection_summary, levels=c("Healthy", "Asymptomatic", "Mild", "Moderate", "Severe", "Critical"))

bcell.model <- model.matrix(~ Sex + Age + OrderedSeverity, data=bcell.meta[bcell.meta$Collection_Day %in% c("D0"), ])


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
# count cells
cell.freq.tab <- t(table(bcell.df$sample_id[bcell.df$Collection_Day %in% c("D0") &
                                                        !bcell.df$Status_on_day_collection_summary %in% c("LPS", "Non_covid")],
                         bcell.df$celltype_B[bcell.df$Collection_Day %in% c("D0") &
                                                        !bcell.df$Status_on_day_collection_summary %in% c("LPS", "Non_covid")]))
cell.freq.tab <- cell.freq.tab[,colSums(cell.freq.tab) != 0]
bcell.model <- bcell.model[colnames(cell.freq.tab), ]

n.cell.vecc <- table(bcell.df$sample_id)

library(edgeR)
bcell.dge <- DGEList(cell.freq.tab, lib.size=log(n.cell.vecc[colnames(cell.freq.tab)]))

Loading required package: limma



In [3]:
#estimate dispersions and fit model
bcell.dge <- estimateDisp(bcell.dge, design=bcell.model)
bcell.linear.fit <- glmQLFit(bcell.dge, bcell.model, robust=TRUE)

In [4]:
# linear changes
bcell.res <- as.data.frame(topTags(glmQLFTest(bcell.linear.fit, coef=4), sort.by='none', n=Inf))
bcell.res$CellType <- rownames(bcell.res)
bcell.res$Sig <- as.numeric(bcell.res$FDR < 0.1)
bcell.res$Diff <- sign(bcell.res$logFC)
bcell.res$Diff[bcell.res$FDR > 0.1] <- 0
bcell.res

,logFC,logCPM,F,PValue,FDR,CellType,Sig,Diff
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
B_exhausted,0.2681812,15.44288,0.930243,3.366870e-01,3.366870e-01,B_exhausted,0,0
B_immature,-0.4307860,16.33239,3.723838,5.593722e-02,8.390583e-02,B_immature,1,-1
B_naive,-0.1142130,19.19977,1.994105,1.604303e-01,2.062676e-01,B_naive,0,0
B_non-switched_memory,-0.2718400,15.41235,1.361181,2.455836e-01,2.762816e-01,B_non-switched_memory,0,0
B_switched_memory,-0.3683969,16.68778,3.934757,4.951903e-02,8.390583e-02,B_switched_memory,1,-1
Plasma_cell_IgA,0.5650620,15.38621,3.724531,5.591471e-02,8.390583e-02,Plasma_cell_IgA,1,1
Plasma_cell_IgG,1.4361268,15.44561,22.229008,6.427078e-06,2.892185e-05,Plasma_cell_IgG,1,1
Plasma_cell_IgM,1.6291556,13.66318,15.593263,1.311839e-04,3.935517e-04,Plasma_cell_IgM,1,1
Plasmablast,2.2128204,15.54715,39.498719,5.153872e-09,4.638484e-08,Plasmablast,1,1


In [5]:
# Quadratic changes.
bcell.quad.res <- as.data.frame(topTags(glmQLFTest(bcell.linear.fit, coef=5), sort.by='none', n=Inf))
bcell.quad.res$CellType <- rownames(bcell.quad.res)
bcell.quad.res$Sig <- as.numeric(bcell.quad.res$FDR < 0.1)
bcell.quad.res$Diff <- sign(bcell.quad.res$logFC)
bcell.quad.res$Diff[bcell.quad.res$FDR > 0.1] <- 0
bcell.quad.res

,logFC,logCPM,F,PValue,FDR,CellType,Sig,Diff
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
B_exhausted,-0.07970675,15.44288,0.1089413,0.7419126350,0.7419126350,B_exhausted,0,0
B_immature,0.29764953,16.33239,2.5311844,0.1141762248,0.2055172047,B_immature,0,0
B_naive,0.09671751,19.19977,1.9598548,0.1640391286,0.2390473922,B_naive,0,0
B_non-switched_memory,0.26123292,15.41235,1.7692781,0.1859257495,0.2390473922,B_non-switched_memory,0,0
B_switched_memory,0.18545452,16.68778,1.3542964,0.2467746542,0.2776214859,B_switched_memory,0,0
Plasma_cell_IgA,-0.69666864,15.38621,7.0716588,0.0088706878,0.0199590476,Plasma_cell_IgA,1,-1
Plasma_cell_IgG,-1.06585835,15.44561,15.2991055,0.0001506300,0.0005438326,Plasma_cell_IgG,1,-1
Plasma_cell_IgM,-1.32080518,13.66318,15.0494302,0.0001694376,0.0005438326,Plasma_cell_IgM,1,-1
Plasmablast,-1.21907633,15.54715,14.9063967,0.0001812775,0.0005438326,Plasmablast,1,-1
